### Open-source Model Enablement with LiteLLM

To implement the usage of LiteLLM and enable AutoGen to use open-sourced models while considering the context provided, we will address the points raised and create a refactoring plan. 

In [ ]:
# ! pip install litellm

In [ ]:
import os
import autogen
import litellm
from litellm import completion
print(autogen.__version__)
from autogen import AssistantAgent, UserProxyAgent

In [ ]:
# Dotenv with OPENAI_API_KEY
config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='../.env',
    model_api_key_map={
        # "gpt-4": "OPENAI_API_KEY",
        "huggingface/mistralai/Mistral-7B-v0.1": "HUGGINGFACE_HUB",
    },
    filter_dict={
        "model": {
            # "gpt-4",
            "huggingface/mistralai/Mistral-7B-v0.1",
        }
    }
)
print(config_list)

coding_assistant = AssistantAgent(
    name="coding_assistant",
    llm_config={
        "request_timeout": 1000,
        "seed": 42,
        "config_list": config_list,
        "temperature": 0.4,
    },
)

coding_runner = UserProxyAgent(
    name="coding_runner",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    # is_termination_msg = lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False
    },
)
coding_runner.initiate_chat(coding_assistant, message="Calculate the percentage gain YTD for Berkshire Hathaway stock.")

*remove OPENAI_API_KEY from .env*

In [ ]:
import autogen
from autogen import AssistantAgent, UserProxyAgent
# config_list_from_dotenv no OPENAI_API_KEY
config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='../.env',
    model_api_key_map={
        "gpt-4": "OPENAI_API_KEY",
        "huggingface/mistralai/Mistral-7B-v0.1": "HUGGINGFACE_HUB",
    },
    filter_dict={
        "model": {
            "gpt-4",
            "huggingface/mistralai/Mistral-7B-v0.1",
        }
    }
)
print(config_list)

coding_assistant = AssistantAgent(
    name="coding_assistant",
    llm_config={
        "request_timeout": 1000,
        "seed": 42,
        "config_list": config_list,
        "temperature": 0.4,
    },
)

coding_runner = UserProxyAgent(
    name="coding_runner",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    is_termination_msg = lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False
    },
)
coding_runner.initiate_chat(coding_assistant, message="Calculate the percentage gain YTD for Berkshire Hathaway stock, save to png.")

In [1]:
import autogen
from autogen import AssistantAgent, UserProxyAgent

config_list = autogen.config_list_from_json(
    env_or_file='OAI_CONFIG_LIST',
    filter_dict={
            "model": ["huggingface/mistralai/Mistral-7B-v0.1"],
        },
    )
print(config_list)

coding_assistant = AssistantAgent(
    name="coding_assistant",
    llm_config={
        "request_timeout": 1000,
        "seed": 42,
        "config_list": config_list,
        "temperature": 0.4,
    },
)

coding_runner = UserProxyAgent(
    name="coding_runner",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=30,
    is_termination_msg = lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False
    },
)
coding_runner.initiate_chat(coding_assistant, message="Calculate the percentage gain YTD for Berkshire Hathaway stock and plot a chart to linechart.png")

[]
coding_runner (to coding_assistant):

Calculate the percentage gain YTD for Berkshire Hathaway stock and plot a chart to linechart.png

--------------------------------------------------------------------------------


AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

#### Model Names vs. Config Names Discrepancy

In [ ]:
print(type(autogen.Completion.chat_models))
print(autogen.Completion.chat_models)

In [ ]:
print(type(autogen.Completion.price1K))
print(autogen.Completion.price1K)

In [ ]:
print(type(litellm.get_model_cost_map()))
print(litellm.get_model_cost_map())

In [ ]:
import json
import difflib
from autogen.oai import Completion

def map_oai_litellm_models():
    """
    The purpose of this function is to find the corresponding
    mappings of oai models as their names are different in litellm,
    which we need to enable building agents with open sources models.

    # Mapping
    # For incommon models, a direct mapping can be created as they are named the same in both systems.
    # For unique models, manual mapping might be needed.
    """

    # Fetching model data
    litellm_models = set(litellm.get_model_cost_map().keys())  # Assuming it's a dict
    autogen_models = set(Completion.price1K)  # Assuming it's an iterable of model names
    
    # Identifying common and unique models
    common_models = autogen_models.intersection(litellm_models)
    unique_autogen_models = autogen_models.difference(litellm_models)
    unique_litellm_models = litellm_models.difference(autogen_models)
    
    # Mapping for common models
    common_mapping = {model: model for model in common_models}
    
    # Mapping for unique models
    unique_mapping = {}
    for auto_model in unique_autogen_models:
        closest_match = difflib.get_close_matches(auto_model, unique_litellm_models, n=2, cutoff=0.1)
        if closest_match:
            unique_mapping[auto_model] = closest_match[0]
        else:
            # Handle unmatched models. Potentially log them for manual mapping
            unique_mapping[auto_model] = None
            print(f"No match found for {auto_model}. Consider manually mapping this model.")
    
    # Merge mappings
    model_mapping = {**common_mapping, **unique_mapping}
    
    return model_mapping

model_mapping = map_oai_litellm_models()

In [ ]:
import pprint as pp

model_mapping.pop('code-cushman-001', None)
pp.pprint(model_mapping)


In [ ]:
# us this to replace names to suit LiteLLM.
litellm.model_alias_map = {
    'code-davinci-002': 'davinci-002',
    'gpt-3.5-turbo': 'gpt-3.5-turbo',
    'gpt-3.5-turbo-0301': 'gpt-3.5-turbo-0301',
    'gpt-3.5-turbo-0613': 'gpt-3.5-turbo-0613',
    'gpt-3.5-turbo-16k': 'gpt-3.5-turbo-16k',
    'gpt-3.5-turbo-16k-0613': 'gpt-3.5-turbo-16k-0613',
    'gpt-35-turbo': 'gpt-3.5-turbo-instruct',
    'gpt-4': 'gpt-4',
    'gpt-4-0314': 'gpt-4-0314',
    'gpt-4-0613': 'gpt-4-0613',
    'gpt-4-32k': 'gpt-4-32k',
    'gpt-4-32k-0314': 'gpt-4-32k-0314',
    'gpt-4-32k-0613': 'gpt-4-32k-0613',
    'text-ada-001': 'text-ada-001',
    'text-babbage-001': 'text-babbage-001',
    'text-curie-001': 'text-curie-001',
    'text-davinci-002': 'davinci-002',
    'text-davinci-003': 'text-davinci-003'
}